In [13]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import numpy as np
import pandas as pd
from pandas.io.parsers import read_csv
from collections import defaultdict
import pickle
from random import sample,random
import copy
from BOAmodel_JMLRv2 import *
from pylab import *
from random import sample
from sklearn.metrics import auc
%matplotlib inline
rcParams['figure.figsize'] = (10, 10)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
def binary_code(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[col].quantile(float(q)/Nlevel)
            df[col+'_geq_'+str(int(q))+'q'] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)

# Generating Simulated Datasets 

In [15]:
Nlevel = 5
Ntrue = 5
for N,J in zip([10000],[50]):
    for iteration in range(100):
        experiment = {}
        print('================== iter = {}, Ntrue = {} =================='.format(iteration,Ntrue))
        df = pd.DataFrame(30*np.random.rand(N,J),columns = ['att'+str(i+1) for i in range(J)])  
        df0 = df.copy(deep = True)
        binary_code(df,df.columns,Nlevel)
        T = np.random.choice([0, 1], size=(N,), p=[0.5, 0.5])

        #=== true set subgroup ====
        # generate true rule set
        flag = True
        while flag: # generate the Y0 via a linear model; Y0 represents the outcome if not receiving any treatment
            w0 = np.random.rand(J) - 0.5
            g0 = np.dot(df0,w0)
            Y0 = (1.0/(1+np.exp(-g0))>=0.5).astype(int)
            if mean(Y0)>=0.3 and mean(Y0)<=0.7:
                flag = False
        BOA_model = BOA(df,1-Y0) # use Bayesian Rule Set to generate rules
        BOA_model.generate_rules(5,3,2000,True,5,'randomforest','precision')
        candidates = []
        threshold = sum(BOA_model.Y)*0.05
        for r in range(BOA_model.RMatrix.shape[1]):
            Z = np.nonzero(BOA_model.RMatrix[:,r])[0]
            if mean(BOA_model.Y[Z])>0.5 and sum(BOA_model.RMatrix[:,r])>threshold:
                candidates.append(r)
        TrueSet = [BOA_model.rules[i] for i in np.random.choice(candidates,Ntrue)]


        dfn = 1-df
        dfn.columns = [name.strip() + 'neg' for name in df.columns]
        df = pd.concat([df,dfn],axis = 1)
        Z = np.zeros(N)
        for rule in TrueSet:
            Z += (np.sum(df[rule],axis=1)==len(rule)).astype(int)
        Z = np.array((Z>0).astype(int))
                
        ind = np.where(np.multiply(T,Z)==1)
        Y = Y0[:]
        Y[ind]=1 # treatment is effective if receiving the treatment as well as being in the true group


        experiment['Z'] = Z # indicates whether an instance belongs to the true subgroup
        experiment['df'] = df # generated dataset
        experiment['Y0'] = Y0 # true Y if not receiving the treatment
        experiment['T'] = T # treatment
        experiment['Y'] = Y # observed Y
        pickle.dump(experiment,open('simulations_'+str(iteration)+'.pkl','wb'))


================== iter = 0, Ntrue = 5 ==================
NEW -- Using random forest to generate rules ...
	Took 12.090s to generate 12924 rules


# Generating Simulated Datasets - Injecting random noise

In [16]:
Nlevel = 5
Ntrue = 5
for N,J in zip([10000],[50]):
    for iteration in range(100):
        experiment = {}
        print('================== iter = {}, Ntrue = {} =================='.format(iteration,Ntrue))
        df = pd.DataFrame(30*np.random.rand(N,J),columns = ['att'+str(i+1) for i in range(J)])  
        df0 = df.copy(deep = True)
        binary_code(df,df.columns,Nlevel)
        T = np.random.choice([0, 1], size=(N,), p=[0.5, 0.5])

        #=== true set subgroup ====
        # generate true rule set
        flag = True
        while flag:
            w0 = np.random.rand(J) - 0.5
            g0 = np.dot(df0,w0)
            Y0 = (1.0/(1+np.exp(-g0))>=0.5).astype(int)
            if mean(Y0)>=0.3 and mean(Y0)<=0.7:
                flag = False
        BOA_model = BOA(df,1-Y0)
        BOA_model.generate_rules(5,3,2000,True,5,'randomforest','precision')
        candidates = []
        threshold = sum(BOA_model.Y)*0.05
        for r in range(BOA_model.RMatrix.shape[1]):
            Z = np.nonzero(BOA_model.RMatrix[:,r])[0]
            if mean(BOA_model.Y[Z])>0.5 and sum(BOA_model.RMatrix[:,r])>threshold:
                candidates.append(r)
        TrueSet = [BOA_model.rules[i] for i in np.random.choice(candidates,Ntrue)]


        dfn = 1-df
        dfn.columns = [name.strip() + 'neg' for name in df.columns]
        df = pd.concat([df,dfn],axis = 1)
        Z = np.zeros(N)
        for rule in TrueSet:
            Z += (np.sum(df[rule],axis=1)==len(rule)).astype(int)
        Z = np.array((Z>0).astype(int))

        # === inserting random noise to the true subgroups ===
        index = sample(range(len(train_index)),int(0.1*len(train_index)))[0]
        Z[index] = 1-Z[index]
        # =======================
        
        ind = np.where(np.multiply(T,Z)==1)
        Y = Y0[:]
        Y[ind]=1

        experiment['Z'] = Z # indicates whether an instance belongs to the true subgroup
        experiment['df'] = df # generated dataset
        experiment['Y0'] = Y0 # true Y if not receiving the treatment
        experiment['T'] = T # treatment
        experiment['Y'] = Y # observed Y
        pickle.dump(experiment,open('simulations_'+str(iteration)+'.pkl','wb'))





================== iter = 0, Ntrue = 5 ==================
NEW -- Using random forest to generate rules ...
	Took 11.716s to generate 13454 rules


# Generating simulated data -  linear subgroups

In [17]:
Nlevel = 5
Ntrue = 5
for N,J in zip([10000],[50]):
    for iteration in range(100):
        experiment = {}
        print('================== iter = {}, Ntrue = {} =================='.format(iteration,Ntrue))
        df = pd.DataFrame(30*np.random.rand(N,J),columns = ['att'+str(i+1) for i in range(J)])  
        df0 = df.copy(deep = True)
        binary_code(df,df.columns,Nlevel)
        T = np.random.choice([0, 1], size=(N,), p=[0.5, 0.5])


        #=== Generate true subgroup: linear sugroup ====
        flag = True
        while flag:
            ws = np.random.rand(J) - 0.5
            gs = np.dot(df0,ws)
            Z = (1.0/(1+np.exp(-gs))>=0.5).astype(int)
            if mean(Z)>=0.2 and mean(Z)<=0.6:
                flag = False

                
        ind = np.where(np.multiply(T,Z)==1)
        Y = Y0[:]
        Y[ind]=1

        dfn = 1-df
        dfn.columns = [name.strip() + 'neg' for name in df.columns]
        df = pd.concat([df.iloc[test_index],dfn],axis = 1)
        Z = np.zeros(N)
        for rule in TrueSet:
            Z += (np.sum(df[rule],axis=1)==len(rule)).astype(int)
        Z = (Z>0).astype(int)



        experiment['Z'] = Z # indicates whether an instance belongs to the true subgroup
        experiment['df'] = df # generated dataset
        experiment['Y0'] = Y0 # true Y if not receiving the treatment
        experiment['T'] = T # treatment
        experiment['Y'] = Y # observed Y
        pickle.dump(experiment,open('simulations_'+str(iteration)+'.pkl','wb'))




================== iter = 0, Ntrue = 5 ==================


# Generating simulated data - Non-linear subgroups

In [18]:
Nlevel = 5
Ntrue = 5
for N,J in zip([10000],[50]):
    for iteration in range(100):
        experiment = {}
        print('================== iter = {}, Ntrue = {} =================='.format(iteration,Ntrue))
        df = pd.DataFrame(30*np.random.rand(N,J),columns = ['att'+str(i+1) for i in range(J)])  
        df0 = df.copy(deep = True)
        binary_code(df,df.columns,Nlevel)
        T = np.random.choice([0, 1], size=(N,), p=[0.5, 0.5])



#=== Generate nonelinear sugroup ====
        flag = True
        while flag:
            for k in range(10):
                if random()>0.5:
                    col = sample(df0.columns.tolist(),1)
                    df0['new'+str(k+1)] = pow(df0[col],sample([2,3],1)[0])
                else:
                    cols = sample(df00.columns.tolist(),2)
                    df0['new'+str(k+1)] = df0[cols[0]]*df0[cols[1]]
        
            ws = np.random.rand(df0.shape[1]) - 0.5
            gs = np.dot(df0,ws)
            Z = (1.0/(1+np.exp(-gs))>=0.7).astype(int)

            if mean(Z)>=0.1 and mean(Z)<=0.5:
                flag = False
# ==================================================
                
        ind = np.where(np.multiply(T,Z)==1)
        Y = Y0[:]
        Y[ind]=1

        dfn = 1-df
        dfn.columns = [name.strip() + 'neg' for name in df.columns]
        df = pd.concat([df.iloc[test_index],dfn],axis = 1)
        Z = np.zeros(N)
        for rule in TrueSet:
            Z += (np.sum(df[rule],axis=1)==len(rule)).astype(int)
        Z = (Z>0).astype(int)



        experiment['Z'] = Z # indicates whether an instance belongs to the true subgroup
        experiment['df'] = df # generated dataset
        experiment['Y0'] = Y0 # true Y if not receiving the treatment
        experiment['T'] = T # treatment
        experiment['Y'] = Y # observed Y
        pickle.dump(experiment,open('simulations_'+str(iteration)+'.pkl','wb'))






================== iter = 0, Ntrue = 5 ==================
